In [1]:
import PyPDF2

# Path to the PDF file
pdf_path = 'delete.pdf'

# Initialize a string to hold the extracted text
text = ""

# Open the PDF file
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # Loop through all the pages and extract text
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()

# Print the extracted text
print(text)


The Kenya protests, widely known by hashtag #RejectFinanceBill2024, are a series of ongoing 
decentralized mass protests in Kenya against tax increases , display of opulence by ministers and 
members of parliament , corruption and demand for accountability.  Following the storming of the Kenyan 
Parliament, president William Ruto reportedly rejected the Bill on 28 June; signed into law the 
"Appropriations Bill 2024" to address the budget shortfall caused by the rejection. Nonetheless, protests 
escalated into riots on July 2, with demons trators now demanding Ruto's resignation.  
In May 2024, the proposed tax increases were heavily criticized by younger Kenyans especially 
concerned with the increase in taxes, and these younger Kenyans have "guided the protests". They 
initially mobilized online using social media platforms like X, T ikTok and Instagram. Young activists 
circulated calls to action, translated the bill into several local languages, used the artificial intelligence 
to

In [2]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(api_key='usekey')

# Function to summarize the text
def summarize_text(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes text."},
            {"role": "user", "content": f"Summarize the following text:\n\n{text}"}
        ],
        max_tokens=150,  # Adjust the length of the summary as needed
        temperature=0.7,  # Adjust the creativity of the summary
    )
    
    # Extract and return the summary
    summary = response.choices[0].message.content.strip()
    return summary

# Sample text to summarize
text = "Your extracted text from the PDF goes here."

# Get the summary of the text
summary = summarize_text(text)

# Print the summary
print("Summary:")
print(summary)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [3]:
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def preprocess_text(text):
    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalnum() and word not in stop_words]

def analyze_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    processed_words = preprocess_text(text)
    
    # Get word frequency
    freq_dist = FreqDist(processed_words)
    
    # Get top 10 most common words
    top_words = freq_dist.most_common(10)
    
    # Basic sentence extraction (for potential answers)
    sentences = sent_tokenize(text)
    
    return {
        'top_words': top_words,
        'sentences': sentences,
        'word_count': len(processed_words),
        'sentence_count': len(sentences)
    }

def answer_question(question, analysis):
    # Simple keyword matching
    question_words = set(preprocess_text(question))
    best_sentence = ''
    max_overlap = 0
    
    for sentence in analysis['sentences']:
        sentence_words = set(preprocess_text(sentence))
        overlap = len(question_words.intersection(sentence_words))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sentence = sentence
    
    return best_sentence if best_sentence else "Sorry, I couldn't find a relevant answer."

# Usage
pdf_path = 'delete.pdf'
analysis = analyze_pdf(pdf_path)

print("Top 10 most common words:")
for word, count in analysis['top_words']:
    print(f"{word}: {count}")

print(f"\nTotal words: {analysis['word_count']}")
print(f"Total sentences: {analysis['sentence_count']}")

# Example question answering
question = "What is the main topic of this document?"
answer = answer_question(question, analysis)
print(f"\nQ: {question}")
print(f"A: {answer}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Top 10 most common words:
protests: 39
police: 37
bill: 33
protesters: 32
kenya: 29
june: 27
people: 21
ruto: 20
nairobi: 18
parliament: 15

Total words: 2200
Total sentences: 170

Q: What is the main topic of this document?
A: 25 June  Main article: 2024 storming of the Kenyan Parliament  
Protesters, numbering in thousands, managed to break through police barricades and entered the 
parliamentary complex.


In [5]:
import PyPDF2
import re
from dateutil import parser
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz

nltk.download('punkt')
nltk.download('stopwords')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
    return text

# Function to preprocess extracted text
def preprocess_text(text):
    text = text.replace('\n', ' ').replace('  ', ' ').strip()
    return text

# Load and preprocess the PDF
pdf_path = 'delete.pdf'  # Make sure this file exists in your working directory
text = extract_text_from_pdf(pdf_path)
text = preprocess_text(text)

# Print the first 500 characters to verify the content
print(text[:500])

# Load the model and tokenizer
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

def normalize_date(date_string):
    try:
        parsed_date = parser.parse(date_string)
        return parsed_date.strftime("%d %B")
    except ValueError:
        return date_string.lower()

def find_relevant_context(text, question):
    sentences = re.split('(?<=[.!?]) +', text)
    
    question_lower = question.lower()
    
    # Extract date from question
    date_pattern = r'\b(\d{1,2})(?:st|nd|rd|th)?\s*(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b'
    date_match = re.search(date_pattern, question_lower)
    
    if date_match:
        normalized_question_date = normalize_date(date_match.group())
        
        # Find sentences with matching dates
        relevant_sentences = []
        for sent in sentences:
            sent_date_match = re.search(date_pattern, sent.lower())
            if sent_date_match:
                normalized_sent_date = normalize_date(sent_date_match.group())
                if normalized_sent_date == normalized_question_date:
                    relevant_sentences.append(sent)
        
        if relevant_sentences:
            context = " ".join(relevant_sentences)
            return context
        else:
            return f"I don't have any information about events on {normalized_question_date}."
    
    # If no date is found, use keyword matching
    keywords = [word for word in question_lower.split() if len(word) > 2]
    relevant_sentences = [
        sent for sent in sentences 
        if any(keyword in sent.lower() for keyword in keywords)
    ]
    
    if not relevant_sentences:
        return f"I'm sorry, I don't have enough information about '{question}' in the given context."
    
    context = " ".join(relevant_sentences)
    
    if len(context.split()) < 50:
        start_index = max(0, sentences.index(relevant_sentences[0]) - 2)
        end_index = min(len(sentences), sentences.index(relevant_sentences[-1]) + 3)
        context = " ".join(sentences[start_index:end_index])
    
    return context

def answer_question(question):
    relevant_context = find_relevant_context(text, question)
    
    # If the context is an "I don't have information" message, return it directly
    if relevant_context.startswith("I don't have") or relevant_context.startswith("I'm sorry"):
        return relevant_context, "Context"
    
    # For date-related queries, return the context directly
    date_pattern = r'\b(\d{1,2})(?:st|nd|rd|th)?\s*(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b'
    if re.search(date_pattern, question.lower()):
        return relevant_context, "Context"
    
    inputs = tokenizer(question, relevant_context, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    answer_start = outputs.start_logits.argmax()
    answer_end = outputs.end_logits.argmax()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
    
    if answer.strip() in ["", "<s>", "</s>"]:
        return relevant_context, "Context"
    
    return answer, relevant_context

# Test questions
test_questions = [
    "7 July",
    "7th July",
    "What happened on 7th July?",
    "Events of 7 July",
    "Tell me about July 7",
    "5 July",
    "5th July",
    "What occurred on 5th July?",
    "Events on July 5",
    "What happened on 5 July?",
    "Describe events of 5th"
]

# Run tests
for question in test_questions:
    answer, context = answer_question(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Context: {context}")
    print("---")

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The Kenya protests, widely known by hashtag #RejectFinanceBill2024, are a series of ongoing decentralized mass protests in Kenya against tax increases , display of opulence by ministers and members of parliament , corruption and demand for accountability. Following the storming of the Kenyan Parliament, president William Ruto reportedly rejected the Bill on 28 June; signed into law the "Appropriations Bill 2024" to address the budget shortfall caused by the rejection. Nonetheless, protests escal


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Question: 7 July
Answer: 7 July  Coinciding with the anniversary of the 1990 Saba Saba Day protests that led to the introduction of multiparty democracy in Kenya, a concert was held in Uhuru Park in Nairobi to commemorate those killed during the protests against the finance bill.
Context: Context
---
Question: 7th July
Answer: 7 July  Coinciding with the anniversary of the 1990 Saba Saba Day protests that led to the introduction of multiparty democracy in Kenya, a concert was held in Uhuru Park in Nairobi to commemorate those killed during the protests against the finance bill.
Context: Context
---
Question: What happened on 7th July?
Answer: 7 July  Coinciding with the anniversary of the 1990 Saba Saba Day protests that led to the introduction of multiparty democracy in Kenya, a concert was held in Uhuru Park in Nairobi to commemorate those killed during the protests against the finance bill.
Context: Context
---
Question: Events of 7 July
Answer: 7 July  Coinciding with the anniversa

In [6]:
from flask import Flask, request, jsonify, render_template
import PyPDF2
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import re
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

app = Flask(__name__)

# QA list
qa_list = {
    "greetings": {
        "questions": ["hello", "hey", "hi", "greetings", "good morning", "good afternoon", "good evening"],
        "answer": "Hello! How can I assist you today?"
    },
    "president_of_kenya": {
        "questions": ["who is the president of kenya", "president of kenya", "kenya's president"],
        "answer": "William Ruto is the President of Kenya."
    },
    "finance_bill_taxes": {
        "questions": [
            "which taxes were introduced in the finance bill",
            "which tax measures did the bill propose",
            "which taxes did the bill introduce",
            "taxes introduced in the finance bill",
            "taxes in the finance bill",
            "what are the new taxes in the finance bill",
            "finance bill tax changes",
            "tax increases in the finance bill"
        ],
        "answer": """The finance bill proposed several measures to raise taxes, which would increase the cost of living:
* Eco Levy: Introduced on imported environmentally harmful products like sanitary towels, diapers, motorcycles, tires, plastic packaging, and electronic equipment.
* 16% VAT: Applied to ordinary bread, sugarcane transportation, locally assembled mobile phones, electric bikes, solar and lithium-ion batteries, and electric buses, changing their tax status from exempt to standard.
* Excise Duty Calculation: Adjusted for alcoholic beverages, cigarettes, and tobacco products, potentially increasing taxes on these items by up to 40% or higher for stronger alcoholic drinks.
* 25% Excise Duty: Imposed on vegetable and seed oils; 5% duty or KES 27,000 per ton on coal, whichever is higher.
* Road Maintenance Levy: Increased from KES 18 to KES 25 per liter of fuel, raising fuel prices.
* 20% Excise Duty: Applied to financial services transactions, telephone and internet services, lottery, betting, gaming, and internet/social media advertisements.
* Motor Vehicle Tax: Introduced a 2.5% tax with a minimum of KES 5,000, later amended to remove the maximum limit of KES 100,000."""
    }
}

def preprocess_question(question):
    stop_words = set(stopwords.words('english'))
    return [word.lower() for word in word_tokenize(question) if word.lower() not in stop_words]

def find_best_match(question, qa_list):
    best_match = None
    best_score = 0
    processed_question = preprocess_question(question)

    for key, qa in qa_list.items():
        for q in qa["questions"]:
            similarity = fuzz.ratio(question.lower(), q.lower())
            processed_q = preprocess_question(q)
            keyword_overlap = len(set(processed_question) & set(processed_q)) / len(set(processed_question) | set(processed_q))
            combined_score = 0.6 * similarity + 0.4 * keyword_overlap * 100

            if combined_score > best_score:
                best_score = combined_score
                best_match = key

    return best_match if best_score > 60 else None

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
    return text

# Function to preprocess extracted text
def preprocess_text(text):
    text = text.replace('\n', ' ').replace('  ', ' ').strip()
    return text

# Load and preprocess the PDF
pdf_path = 'delete.pdf'
text = extract_text_from_pdf(pdf_path)
text = preprocess_text(text)

# Load the model and tokenizer
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Function to find relevant context for a question
def find_relevant_context(text, question):
    sentences = re.split('(?<=[.!?]) +', text)
    
    question_lower = question.lower()
    
    if question_lower.startswith("who is"):
        name_or_title = question_lower.replace("who is", "").strip()
    else:
        name_or_title = question_lower
    
    keywords = name_or_title.split()
    
    titles = ["president", "minister", "governor", "senator", "mp", "activist", "journalist", "inspector general", "police"]
    keywords.extend(titles)
    
    relevant_sentences = [
        sent for sent in sentences 
        if any(word.lower() in sent.lower() for word in keywords)
    ]
    
    if not relevant_sentences:
        return text
    
    context = " ".join(relevant_sentences)
    
    if len(context.split()) < 50:
        start_index = max(0, sentences.index(relevant_sentences[0]) - 2)
        end_index = min(len(sentences), sentences.index(relevant_sentences[-1]) + 3)
        context = " ".join(sentences[start_index:end_index])
    
    return context

# Function to answer questions based on context
def answer_question_with_context(question, context, tokenizer, model):
    inputs = tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    answer_start = outputs.start_logits.argmax()
    answer_end = outputs.end_logits.argmax()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
    
    if answer.strip() in ["", "<s>", "</s>"]:
        name = question.lower().replace("who is", "").strip()
        sentences = re.split('(?<=[.!?]) +', context)
        
        for sentence in sentences:
            if name in sentence.lower():
                title_match = re.search(r'(\w+\s+\w+)\s+' + re.escape(name), sentence, re.IGNORECASE)
                if title_match:
                    return f"{name.title()} is {title_match.group(1)}", context
                else:
                    return sentence, context
        
        return f"I'm sorry, I don't have enough information about {name} in the given context.", context
    
    return answer, context

# Function to answer questions
def answer_question(question):
    best_match = find_best_match(question, qa_list)
    if best_match:
        return qa_list[best_match]["answer"], "QA List"

    relevant_context = find_relevant_context(text, question)
    answer, used_context = answer_question_with_context(question, relevant_context, tokenizer, model)
    return answer, used_context

# Example usage
questions = [
    "hello",
    "who is the president of kenya",
    "taxes in the new finance bill?",
    "which taxes were introduced",
]

for question in questions:
    answer, context = answer_question(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Context: {context}\n")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Question: hello
Answer: Hello! How can I assist you today?
Context: QA List

Question: who is the president of kenya
Answer: William Ruto is the President of Kenya.
Context: QA List

Question: taxes in the new finance bill?
Answer: The finance bill proposed several measures to raise taxes, which would increase the cost of living:
* Eco Levy: Introduced on imported environmentally harmful products like sanitary towels, diapers, motorcycles, tires, plastic packaging, and electronic equipment.
* 16% VAT: Applied to ordinary bread, sugarcane transportation, locally assembled mobile phones, electric bikes, solar and lithium-ion batteries, and electric buses, changing their tax status from exempt to standard.
* Excise Duty Calculation: Adjusted for alcoholic beverages, cigarettes, and tobacco products, potentially increasing taxes on these items by up to 40% or higher for stronger alcoholic drinks.
* 25% Excise Duty: Imposed on vegetable and seed oils; 5% duty or KES 27,000 per ton on coal, 

In [7]:
import PyPDF2
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import re
import nltk

nltk.download('punkt')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
    return text

# Function to preprocess extracted text
def preprocess_text(text):
    text = text.replace('\n', ' ').replace('  ', ' ').strip()
    return text

# Load and preprocess the PDF
pdf_path = 'delete.pdf'
text = extract_text_from_pdf(pdf_path)
text = preprocess_text(text)

# Load the model and tokenizer
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Function to find relevant context for a question
def find_relevant_context(text, question):
    sentences = re.split('(?<=[.!?]) +', text)
    
    question_lower = question.lower()
    
    keywords = question_lower.split()
    
    relevant_sentences = [
        sent for sent in sentences 
        if any(word.lower() in sent.lower() for word in keywords)
    ]
    
    if not relevant_sentences:
        return text
    
    context = " ".join(relevant_sentences)
    
    if len(context.split()) < 50:
        start_index = max(0, sentences.index(relevant_sentences[0]) - 2)
        end_index = min(len(sentences), sentences.index(relevant_sentences[-1]) + 3)
        context = " ".join(sentences[start_index:end_index])
    
    return context

# Function to answer questions based on context
def answer_question_with_context(question, context, tokenizer, model):
    inputs = tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    answer_start = outputs.start_logits.argmax()
    answer_end = outputs.end_logits.argmax()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
    
    if answer.strip() in ["", "<s>", "</s>"]:
        return "I'm sorry, I don't have enough information to answer that question.", context
    
    return answer, context

# Function to answer questions
def answer_question(question):
    relevant_context = find_relevant_context(text, question)
    answer, used_context = answer_question_with_context(question, relevant_context, tokenizer, model)
    return answer, used_context

# Example usage
questions = [
    "When did the protest start?",
    "How did people mobilize for the protest?",
    "Japheth Koome"
]

for question in questions:
    answer, context = answer_question(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Question: When did the protest start?
Answer:  18 June
Question: How did people mobilize for the protest?
Answer:  online using social media platforms like X, T ikTok and Instagram
Question: Japheth Koome
Answer: I'm sorry, I don't have enough information to answer that question.


In [8]:
import PyPDF2
import re

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
    return text

# Function to preprocess extracted text
def preprocess_text(text):
    text = text.replace('\n', ' ').replace('  ', ' ').strip()
    return text

# Function to find the sentence containing a specific term
def find_sentence_with_term(text, term):
    sentences = re.split('(?<=[.!?]) +', text)
    for sentence in sentences:
        if term.lower() in sentence.lower():
            return sentence
    return "I'm sorry, I don't have enough information to answer that question."

# Load and preprocess the PDF
pdf_path = 'delete.pdf'
text = extract_text_from_pdf(pdf_path)
text = preprocess_text(text)

# Search for the term "Japheth Koome"
term = "Japheth Koome"
answer = find_sentence_with_term(text, term)

# Print the question and answer
print(f"Question: {term}")
print(f"Answer: {answer}")

Question: Japheth Koome
Answer: Japheth Koome Nchebere resigned as Inspector -general of Police amid the outcry over authorities' response to the protests.


In [9]:
import PyPDF2
import re

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
    return text

# Function to preprocess extracted text
def preprocess_text(text):
    text = text.replace('\n', ' ').replace('  ', ' ').strip()
    return text

# Function to find the sentence containing a specific term
def find_sentence_with_term(text, term):
    sentences = re.split('(?<=[.!?]) +', text)
    for sentence in sentences:
        if term.lower() in sentence.lower():
            return sentence
    return "I'm sorry, I don't have enough information to answer that question."

# Function to handle different query types
def handle_query(text, query):
    # Define patterns for common queries
    patterns = [
        r"Who is (.+)\?",
        r"Tell me about (.+)",
        r"What can you tell me about (.+)"
    ]
    
    # Search for the term in the query using the patterns
    for pattern in patterns:
        match = re.match(pattern, query, re.IGNORECASE)
        if match:
            term = match.group(1)
            return find_sentence_with_term(text, term)
    
    # Default response if no common query type is found
    return find_sentence_with_term(text, query)

# Load and preprocess the PDF
pdf_path = 'delete.pdf'
text = extract_text_from_pdf(pdf_path)
text = preprocess_text(text)

# Example queries
queries = [
    "Who is Macharia Gaitho?",
    "Tell me about Japheth Koome",
    "What can you tell me about Japheth Koome"
]

# Print the questions and answers
for query in queries:
    answer = handle_query(text, query)
    print(f"Question: {query}")
    print(f"Answer: {answer}\n")


Question: Who is Macharia Gaitho?
Answer: These include the abduction of veteran journalist, Macharia Gaitho a t the Karen police station.

Question: Tell me about Japheth Koome
Answer: Japheth Koome Nchebere resigned as Inspector -general of Police amid the outcry over authorities' response to the protests.

Question: What can you tell me about Japheth Koome
Answer: Japheth Koome Nchebere resigned as Inspector -general of Police amid the outcry over authorities' response to the protests.

